<a href="https://colab.research.google.com/github/AlfredWGA/AnimalGame/blob/master/ner_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. It cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type or occurs at the beginning of a sentence. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? Define _span_ as a sequence of tags that starts with a `B-TYPE` followed by zero or more `I-TYPE` tags. Sequences solely of `I-TYPE` tags don't count as spans.For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [1]:
!pip install --upgrade spacy==2.1.0 allennlp==0.9.0
import spacy

     |████████████████████████████████| 27.7MB 107kB/s 
     |████████████████████████████████| 7.6MB 29.4MB/s 
     |████████████████████████████████| 92kB 12.0MB/s 
     |████████████████████████████████| 3.2MB 55.8MB/s 
     |████████████████████████████████| 2.1MB 46.1MB/s 
     |████████████████████████████████| 5.6MB 50.8MB/s 
     |████████████████████████████████| 163kB 60.2MB/s 
     |████████████████████████████████| 133kB 53.0MB/s 
     |████████████████████████████████| 245kB 56.7MB/s 
     |████████████████████████████████| 133kB 55.0MB/s 
     |████████████████████████████████| 317kB 51.6MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 266kB 57.4MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 256kB 52.6MB/s 
     |████████████████████████████████| 1.2MB 46.6MB/s 
     |████████████████████████████████| 7.4MB 40.1MB/s 

In [2]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [3]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [4]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

from itertools import chain
vocab = Vocabulary.from_instances(chain(train_dataset, validation_dataset))

159377B [00:00, 38424762.24B/s]
562it [00:00, 4464.95it/s]
8366B [00:00, 19049699.93B/s]
23it [00:00, 7337.16it/s]
585it [00:00, 61200.93it/s]


## Define and Train Model

In [5]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9086 ||: 100%|██████████| 281/281 [00:01<00:00, 199.73it/s]
accuracy: 0.7878, loss: 1.2066 ||: 100%|██████████| 12/12 [00:00<00:00, 436.59it/s]
accuracy: 0.8442, loss: 0.7298 ||: 100%|██████████| 281/281 [00:01<00:00, 257.74it/s]
accuracy: 0.7878, loss: 1.1845 ||: 100%|██████████| 12/12 [00:00<00:00, 477.72it/s]
accuracy: 0.8442, loss: 0.7167 ||: 100%|██████████| 281/281 [00:01<00:00, 266.71it/s]
accuracy: 0.7878, loss: 1.1779 ||: 100%|██████████| 12/12 [00:00<00:00, 470.38it/s]
accuracy: 0.8442, loss: 0.7069 ||: 100%|██████████| 281/281 [00:01<00:00, 265.98it/s]
accuracy: 0.7878, loss: 1.1651 ||: 100%|██████████| 12/12 [00:00<00:00, 470.19it/s]
accuracy: 0.8442, loss: 0.6993 ||: 100%|██████████| 281/281 [00:01<00:00, 256.11it/s]
accuracy: 0.7878, loss: 1.1714 ||: 100%|██████████| 12/12 [00:00<00:00, 450.96it/s]
accuracy: 0.8442, loss: 0.6912 ||: 100%|██████████| 281/281 [00:01<00:00, 237.07it/s]
accuracy: 0.7878, loss: 1.1546 ||: 100%|██████████| 12/12 [00:00

{'best_epoch': 99,
 'best_validation_accuracy': 0.8795918367346939,
 'best_validation_loss': 0.3832807225893096,
 'epoch': 99,
 'peak_cpu_memory_MB': 3223.36,
 'peak_gpu_0_memory_MB': 1060,
 'training_accuracy': 0.9253257070225611,
 'training_cpu_memory_MB': 3223.36,
 'training_duration': '0:02:01.533069',
 'training_epochs': 99,
 'training_gpu_0_memory_MB': 1060,
 'training_loss': 0.19348942842545797,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8795918367346939,
 'validation_loss': 0.3832807225893096}

## Evaluation

The simple code below loops over the validation set, applying the model to each exmaple and collecting out the input token, gold-standard output, and model output. You can see from these methods how to access ground truth and model outputs for evaluation.

In [6]:
def tag_sentence(s):
  tag_ids = np.argmax(model.forward_on_instance(s)['tag_logits'], axis=-1)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]
## Show the first example
baseline_output[0]

[(With, 'O', 'O'),
 (a, 'O', 'O'),
 (wave, 'O', 'O'),
 (of, 'O', 'O'),
 (his, 'O', 'O'),
 (hand, 'O', 'O'),
 (,, 'O', 'O'),
 (Peng, 'B-PERSON', 'B-PERSON'),
 (Dehuai, 'I-PERSON', 'I-ORG'),
 (said, 'O', 'O'),
 (that, 'O', 'O'),
 (despite, 'O', 'O'),
 (being, 'O', 'O'),
 (over, 'O', 'O'),
 (100, 'B-CARDINAL', 'B-CARDINAL'),
 (regiments, 'O', 'O'),
 (,, 'O', 'O'),
 (let, 'O', 'O'),
 ('s, 'O', 'O'),
 (call, 'O', 'O'),
 (this, 'O', 'O'),
 (campaign, 'O', 'O'),
 (the, 'B-EVENT', 'O'),
 (Hundred, 'I-EVENT', 'I-EVENT'),
 (Regiments, 'I-EVENT', 'I-EVENT'),
 (Offensive, 'I-EVENT', 'I-EVENT'),
 (., 'O', 'O')]

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [7]:
!pip install seqeval
import seqeval

     |████████████████████████████████| 51kB 6.8MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=511d719b628767344c86acdc386658216014278d490dccf19b4c303e08f8229e
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [26]:
# TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input
def violations(tagged):
  count = 0
  for sentence in tagged:
    for i, t in enumerate(sentence):
      t_pred = t[2]
      if t_pred != 'O':
        t1, t2 = t_pred.split('-')
        if t1 == 'I':
          # if the I-TYPE tag follows B-TYPE or I-TYPE 
          if i != 0 and (sentence[i - 1][2] == f'B-{t2}' or sentence[i - 1][2] == t_pred):
            pass
          else:
            count += 1
  return count

# TODO: return the span-level precision, recall, and F1
# Only count valid spans that start with a B tag,
# followed by zero or more I tags of the same type.
# This is harsher than the token-level metric that the
# LSTM was trained to optimize, but it is the standard way
# of evaluating NER systems.
# Take tagger output as input
from typing import List, Tuple
from seqeval.metrics import precision_score, recall_score, f1_score

def span_stats(y_true, y_pred):
  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)
  
  return {'precision': precision,
          'recall': recall,
          'f1': f1}
## You can check how many violations are made by the model output in predictor.
violations(baseline_output)

32

In [27]:
# print(classification_report([[t[1] for t in x] for x in baseline_output], [[t[2] for t in x] for x in baseline_output]))
y_true = [[t[1] for t in x] for x in baseline_output]
y_pred = [[t[2] for t in x] for x in baseline_output]
print(span_stats(y_true, y_pred))

{'precision': 0.16393442622950818, 'recall': 0.23255813953488372, 'f1': 0.1923076923076923}


## Decoding

Now you can finally implement the simple Viterbi decoder. The `model` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the expression `model.forward_on_instance(s)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the model object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [12]:
y_pred_list = [model.forward_on_instance(s)['tag_logits'] for s in validation_dataset]

In [13]:
print(y_pred_list[0].shape)
print(y_pred_list[10].shape)

(27, 34)
(21, 34)


In [14]:
# This code shows how to map from output vector components to labels
print(vocab.get_index_to_token_vocabulary('labels'))

{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [15]:
i2v = vocab.get_index_to_token_vocabulary('labels')
vocab_size = len(i2v)
vocab_size

34

In [16]:
# Construct the transition matrix
M = np.full(shape=[vocab_size, vocab_size], fill_value=-np.inf)
for i in range(vocab_size):
  for j in range(vocab_size):
    if i2v[i].startswith('I'):
      _, token_type = i2v[i].split('-')
      # I-TYPE1 cannot transit to I-TYPE2
      if i2v[j].startswith('I') and i2v[j].split('-')[1] != token_type:
        pass
      else:
        M[i][j] = 0
    if i2v[i].startswith('B'):
      _, token_type = i2v[i].split('-')
      # B-TYPE1 cannot transit to I-TYPE2 
      if i2v[j].startswith('I') and i2v[j].split('-')[1] != token_type:
        pass
      else:
        M[i][j] = 0
    if i2v[i].startswith('O'):
      # O cannot transit to I
      if i2v[j].startswith('I'):
        pass
      else:
        M[i][j] = 0

In [17]:
print(M.shape)
print(M)

(34, 34)
[[  0.   0. -inf ... -inf   0. -inf]
 [  0.   0. -inf ... -inf   0. -inf]
 [  0.   0.   0. ... -inf   0. -inf]
 ...
 [  0.   0. -inf ...   0.   0. -inf]
 [  0.   0. -inf ... -inf   0.   0.]
 [  0.   0. -inf ... -inf   0.   0.]]


In [18]:
v2i = {v: k for (k, v) in i2v.items()}
print(v2i)

{'O': 0, 'B-GPE': 1, 'I-ORG': 2, 'I-DATE': 3, 'B-CARDINAL': 4, 'I-EVENT': 5, 'B-PERSON': 6, 'B-NORP': 7, 'B-DATE': 8, 'B-ORG': 9, 'B-LOC': 10, 'I-LOC': 11, 'I-FAC': 12, 'I-PERSON': 13, 'I-GPE': 14, 'I-CARDINAL': 15, 'B-EVENT': 16, 'I-TIME': 17, 'I-WORK_OF_ART': 18, 'B-ORDINAL': 19, 'B-FAC': 20, 'B-TIME': 21, 'I-LAW': 22, 'I-QUANTITY': 23, 'I-NORP': 24, 'I-MONEY': 25, 'B-MONEY': 26, 'B-WORK_OF_ART': 27, 'B-QUANTITY': 28, 'B-LAW': 29, 'B-PRODUCT': 30, 'I-PRODUCT': 31, 'B-PERCENT': 32, 'I-PERCENT': 33}


In [19]:
start_prob = np.full(shape=[vocab_size,], fill_value=0.0)
for i in range(len(start_prob)):
  if i2v[i].startswith("I"):
    start_prob[i] = -np.inf

# for i, sentence in enumerate(baseline_output):
#   start_token = sentence[0][-1]
#   token_idx = v2i[start_token]
#   prob = y_pred[i][0][token_idx]
#   start_prob[token_idx] += prob

# for i in range(start_prob.shape[0]):
#   if start_prob[i] == 0:
#     start_prob[i] = -np.inf

In [20]:
start_prob

array([  0.,   0., -inf, -inf,   0., -inf,   0.,   0.,   0.,   0.,   0.,
       -inf, -inf, -inf, -inf, -inf,   0., -inf, -inf,   0.,   0.,   0.,
       -inf, -inf, -inf, -inf,   0.,   0.,   0.,   0.,   0., -inf,   0.,
       -inf])

In [21]:
def viterbi_decode(y_pred: np.array, start_prob: np.array, M: np.array, i2v: dict):
  """
  y_pred: array of shape [length, vocab_size]
  start_prob: array of shape [vocab_size,]
  M: the transition matrix of shape [vocab_size, vocab_size]

  """
  length = y_pred.shape[0]
  vocab_size = M.shape[0]
  
  # shape [length, vocab_size], where each element is the probability of the i-th time getting j-th token
  prob_matrix = np.zeros(shape=[length, vocab_size])
  # shape [length, vocab_size], where each element represents the previous token id
  path_matrix = np.zeros(shape=[length, vocab_size], dtype=np.int32)
  for i in range(length):
    if i == 0:
      prob_matrix[i] = start_prob + y_pred[i]
      # print(prob_matrix[i])
      continue
    for j in range(vocab_size):
      # Probability of previous state, and probability of each state transition to state j        
      tran_prob = prob_matrix[i - 1] + M[:, j]
      max_tran_prob = np.max(tran_prob) + y_pred[i][j]
      previous_state = np.argmax(tran_prob)
      # if i == 1 and j == 1:
        # print(tran_prob)
        # print(max_tran_prob)
        # print(previous_state)
      prob_matrix[i][j] = max_tran_prob
      path_matrix[i][j] = previous_state
  
  # return prob_matrix, path_matrix
  paths = np.zeros(shape=[vocab_size, length], dtype=np.int32)
  path_probs = np.zeros(shape=[vocab_size,])
  for j in range(vocab_size):  # For the j-th path
    prob = prob_matrix[-1][j] # Starting from the last time step
    former_idx = path_matrix[-1][j]
    # print(former_idx)
    path = [former_idx, j]
    for i in range(length - 2, 0, -1): # Time step length - 2 to 1
      former_idx = path_matrix[i][former_idx]
      path = [former_idx] + path
      prob += prob_matrix[i][former_idx]
    path = np.asarray(path)
    # print(path.shape)
    paths[j] = path
    path_probs[j] = prob
  
  best_path, best_prob = max(zip(paths, path_probs), key=lambda x: x[1])
  best_path = [i2v[i] for i in best_path]
  return best_path

In [22]:
# prob_matrix, path_matrix = viterbi_decode(y_pred_list[0], start_prob, M)
best_path = viterbi_decode(y_pred_list[0], start_prob, M, i2v)

In [23]:
best_path

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PERSON',
 'I-PERSON',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-CARDINAL',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-EVENT',
 'I-EVENT',
 'I-EVENT',
 'I-EVENT',
 'O']

In [24]:
viterbi_outputs = []
for i, y_pred in enumerate(y_pred_list):
  viterbi_outputs.append(viterbi_decode(y_pred, start_prob, M, i2v))

In [31]:
print(span_stats(y_true, viterbi_outputs))

{'precision': 0.4318181818181818, 'recall': 0.4418604651162791, 'f1': 0.4367816091954023}


In [34]:
print('Number of violations after using Viterbi algorithm:', violations([[(token[0], token[1], viterbi_outputs[i][j]) for j, token in enumerate(sentence)] for i, sentence in enumerate(baseline_output)]))

Number of violations after using Viterbi algorithm: 0
